# Neural Network Testing with Regular Data

### Get Libraries and Functions

In [1]:
import sys 
sys.path.append("../")

import torch 
import torch.nn

from Conv1d_NN import Conv1d_NN

from models import * 
from train import * 
from dataset import * 



The Arguments for Data are: 
num_samples: 5000 
 train_split: 0.8 
 template_len: 12 
 padding: [36,60] 
 scale_coeff: .4 
 max_translation: 48 
 corr_noise_scale: 0.25 
 iid_noise_scale: 2e-2 
 shear_scale: 0.75 
 shuffle_seq: False 
 final_seq_length: 40 
 seed: 42


The Arguments for Model are: 
input_size: 40 
 output_size: 10 
 hidden_size: 256 
 learning_rate: 1e-2 
 weight_decay: 0 
 batch_size: 100 
 total_steps: 6000 
 print_every: 1000 
 eval_every: 250 
 checkpoint_every: 1000 
 device: mps 
 seed: 42


### Regular Data

In [2]:
# Regular Noise Dataset
dataset = MNIST1D_Dataset() 
data = dataset.make_dataset()

The Arguments for Data are: 
num_samples: 5000 
 train_split: 0.8 
 template_len: 12 
 padding: [36,60] 
 scale_coeff: .4 
 max_translation: 48 
 corr_noise_scale: 0.25 
 iid_noise_scale: 2e-2 
 shear_scale: 0.75 
 shuffle_seq: False 
 final_seq_length: 40 
 seed: 42


The Arguments for Model are: 
input_size: 40 
 output_size: 10 
 hidden_size: 256 
 learning_rate: 1e-2 
 weight_decay: 0 
 batch_size: 100 
 total_steps: 6000 
 print_every: 1000 
 eval_every: 250 
 checkpoint_every: 1000 
 device: mps 
 seed: 42


### Regular Conv1d_NN

In [13]:
# Regular Conv1d_NN
model1 = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =32, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1280, 10)
)

# Get summary 
from torchsummary import summary
summary(model1, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 40]           3,104
         Conv1d_NN-5               [-1, 32, 40]               0
              ReLU-6               [-1, 32, 40]               0
            Conv1d-7               [-1, 32, 40]           3,104
         Conv1d_NN-8               [-1, 32, 40]               0
              ReLU-9               [-1, 32, 40]               0
          Flatten-10                 [-1, 1280]               0
           Linear-11                   [-1, 10]          12,810
Total params: 19,146
Trainable params: 19,146
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [14]:
# Regular Conv1d_NN Results 
print("Regular Conv1d_NN Results")
results1 = train_model(data, model1, dataset.model_args)

Regular Conv1d_NN Results
step 1000, dt 18.41s, train_loss 9.264e-01, test_loss 1.584e+00, train_acc 59.7, test_acc 43.2
step 2000, dt 18.25s, train_loss 7.961e-01, test_loss 1.691e+00, train_acc 69.5, test_acc 44.8
step 3000, dt 18.16s, train_loss 7.664e-01, test_loss 1.799e+00, train_acc 71.2, test_acc 45.8
step 4000, dt 18.43s, train_loss 7.092e-01, test_loss 1.988e+00, train_acc 70.9, test_acc 44.8
step 5000, dt 17.80s, train_loss 7.080e-01, test_loss 1.988e+00, train_acc 72.5, test_acc 45.7
step 6000, dt 18.03s, train_loss 6.511e-01, test_loss 2.207e+00, train_acc 75.0, test_acc 42.8


### Local -> Global Neural Network

In [15]:
# Local -> Global Conv1d_NN
model2 = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model2, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 38]             128
              ReLU-2               [-1, 32, 38]               0
            Conv1d-3               [-1, 32, 38]           3,104
         Conv1d_NN-4               [-1, 32, 38]               0
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 36]           3,104
              ReLU-7               [-1, 32, 36]               0
            Conv1d-8               [-1, 32, 36]           3,104
         Conv1d_NN-9               [-1, 32, 36]               0
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 34]           3,104
             ReLU-12               [-1, 32, 34]               0
           Conv1d-13               [-1, 32, 34]           3,104
        Conv1d_NN-14               [-1,

In [16]:
# Local -> Global Conv1d_NN Results 
print("Local -> Global Conv1d_NN Results")
results2 = train_model(data, model2, dataset.model_args)

Local -> Global Conv1d_NN Results
step 1000, dt 21.12s, train_loss 8.296e-01, test_loss 8.875e-01, train_acc 66.8, test_acc 66.8
step 2000, dt 17.73s, train_loss 5.403e-01, test_loss 6.507e-01, train_acc 79.0, test_acc 78.5
step 3000, dt 19.66s, train_loss 3.207e-01, test_loss 6.218e-01, train_acc 83.3, test_acc 79.9
step 4000, dt 21.24s, train_loss 3.450e-01, test_loss 6.499e-01, train_acc 84.3, test_acc 78.9
step 5000, dt 20.92s, train_loss 3.038e-01, test_loss 6.234e-01, train_acc 86.2, test_acc 81.5
step 6000, dt 20.79s, train_loss 4.818e-01, test_loss 5.979e-01, train_acc 83.8, test_acc 81.6


### Global -> Local Neural Network

In [17]:
# Global -> Local Conv1d_NN
model3 = nn.Sequential(
   Conv1d_NN(in_channels=1, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model3, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 38]           3,104
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 38]           3,104
         Conv1d_NN-7               [-1, 32, 38]               0
              ReLU-8               [-1, 32, 38]               0
            Conv1d-9               [-1, 32, 36]           3,104
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 36]           3,104
        Conv1d_NN-12               [-1, 32, 36]               0
             ReLU-13               [-1, 32, 36]               0
           Conv1d-14               [-1,

In [18]:
# Global -> Local Conv1d_NN Results 
print("Global -> Local Conv1d_NN Results")
results3 = train_model(data, model3, dataset.model_args)

Global -> Local Conv1d_NN Results
step 1000, dt 20.98s, train_loss 5.322e-01, test_loss 6.294e-01, train_acc 82.2, test_acc 80.4
step 2000, dt 18.20s, train_loss 3.672e-01, test_loss 5.075e-01, train_acc 87.9, test_acc 84.4
step 3000, dt 17.34s, train_loss 2.868e-01, test_loss 5.910e-01, train_acc 87.9, test_acc 85.0
step 4000, dt 18.95s, train_loss 2.641e-01, test_loss 5.247e-01, train_acc 92.1, test_acc 86.5
step 5000, dt 20.53s, train_loss 2.766e-01, test_loss 5.013e-01, train_acc 93.0, test_acc 88.3
step 6000, dt 20.59s, train_loss 1.514e-01, test_loss 5.077e-01, train_acc 93.0, test_acc 87.8


### Branching Neural Network

In [3]:
model4 = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 3), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 3),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 3),
   nn.Flatten(), 
   nn.Linear(160, 10)
)


# Get summary
from torchsummary import summary 
summary(model4, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 38]              64
              ReLU-2               [-1, 16, 38]               0
            Conv1d-3               [-1, 16, 40]              64
         Conv1d_NN-4               [-1, 16, 40]               0
              ReLU-5               [-1, 16, 40]               0
            Conv1d-6               [-1, 16, 40]             528
  BranchingNetwork-7               [-1, 16, 40]               0
            Conv1d-8                [-1, 8, 38]             392
              ReLU-9                [-1, 8, 38]               0
           Conv1d-10                [-1, 8, 40]             392
        Conv1d_NN-11                [-1, 8, 40]               0
             ReLU-12                [-1, 8, 40]               0
           Conv1d-13                [-1, 8, 40]             136
 BranchingNetwork-14                [-1

In [4]:
# Branching Network Results 
print("Branching Conv1d_NN Conv1d Results")
results4 = train_model(data, model4, dataset.model_args)

Branching Conv1d_NN Conv1d Results
step 1000, dt 24.27s, train_loss 1.688e-01, test_loss 4.230e-01, train_acc 91.6, test_acc 87.0
step 2000, dt 23.49s, train_loss 5.882e-02, test_loss 3.366e-01, train_acc 95.8, test_acc 90.7
step 3000, dt 23.18s, train_loss 7.708e-02, test_loss 2.817e-01, train_acc 96.9, test_acc 91.9
step 4000, dt 23.14s, train_loss 4.215e-02, test_loss 3.680e-01, train_acc 97.4, test_acc 92.2
step 5000, dt 23.26s, train_loss 2.322e-02, test_loss 3.760e-01, train_acc 97.8, test_acc 92.3
step 6000, dt 23.58s, train_loss 5.297e-02, test_loss 3.929e-01, train_acc 98.1, test_acc 93.3


### 